In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from haversine import haversine
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/BR_SOTF_request_by_DMA_20200109'

In [2]:
df_all_store=pd.ExcelFile("./Store List with Retro SOTF highlighted 1-8-20.xlsx")
df_all_store=df_all_store.parse("Sheet1",skiprows=5,dtype=str)
df_all_store=df_all_store[['Location','Zip Code','SOTF']]
df_all_store['zip_cd']=df_all_store['Zip Code'].apply(lambda x: x.split("-")[0].zfill(5))

print(df_all_store.shape,df_all_store['Location'].nunique())
df_SOTF=df_all_store[df_all_store['SOTF']=="YES"]
df_SOTF.shape,df_SOTF['Location'].nunique()

(1424, 4) 1424


((466, 4), 466)

In [3]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [4]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [5]:
df_zip_by_store['zip_cd'].nunique()

12980

In [6]:
df_DMA_mapping=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",
                             skiprows=1,dtype=str)
df_DMA_mapping=df_DMA_mapping.iloc[:,[0,2]]
df_DMA_mapping.columns=['zip_cd','DMA']
print(df_DMA_mapping.shape)
df_DMA_mapping=df_DMA_mapping.drop_duplicates()
print(df_DMA_mapping.shape)


(51324, 2)
(43102, 2)


In [7]:
df_store_location=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt",
                             sep="|",dtype=str)
df_store_location=df_store_location[['location_id','zip_cd','latitude_meas','longitude_meas']]
df_store_location['zip_cd']=df_store_location['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_location['latitude_meas']=df_store_location['latitude_meas'].astype(float)
df_store_location['longitude_meas']=df_store_location['longitude_meas'].astype(float)

df_store_lat_lng=df_store_location.loc[:,['location_id','latitude_meas','longitude_meas']]

In [8]:
dict_location_google={}
# 5405 FRANKFORT	9	REGION 9	998	NEW STORES	1300 US HIGHWAY 127	FRANKFORT	FRANKFORT	KY	40601-4395
dict_location_google.update({"5405":{"GoogleAddress":"1300 US-127, Frankfort, KY 40601","Google_Coor":[38.1668832,-84.900875]}})

# 5412 KETTERING	9	REGION 9	998	NEW STORES	2120 E DOROTHY LN	KETTERING	KETTERING	OH	45420-1114
dict_location_google.update({"5412":{"GoogleAddress":"2120 E Dorothy Ln, Kettering, OH 45420","Google_Coor":[39.7016113,-84.1263975]}})

# 5413 FLORENCE	9	REGION 9	998	NEW STORES	340 SEVILLE ST	FLORENCE	FLORENCE	AL	35630-1511
dict_location_google.update({"5413":{"GoogleAddress":"350 Seville St, Florence, AL 35630","Google_Coor":[34.8381593,-87.6269989]}}) # the next door location

# 5414 BUFORD	9	REGION 9	998	NEW STORES	4125 HIGHWAY 20, STE A-2	BUFORD	BUFORD	GA	30518-3900
dict_location_google.update({"5414":{"GoogleAddress":"4125 GA-20, Buford, GA 30518","Google_Coor":[34.0936214,-84.0007314]}})


In [9]:
dict_location_google

{'5405': {'GoogleAddress': '1300 US-127, Frankfort, KY 40601',
  'Google_Coor': [38.1668832, -84.900875]},
 '5412': {'GoogleAddress': '2120 E Dorothy Ln, Kettering, OH 45420',
  'Google_Coor': [39.7016113, -84.1263975]},
 '5413': {'GoogleAddress': '350 Seville St, Florence, AL 35630',
  'Google_Coor': [34.8381593, -87.6269989]},
 '5414': {'GoogleAddress': '4125 GA-20, Buford, GA 30518',
  'Google_Coor': [34.0936214, -84.0007314]}}

In [10]:
print(df_store_lat_lng.shape,df_store_lat_lng['location_id'].nunique())
for k,v in dict_location_google.items():
    df=pd.DataFrame({"location_id":[k],"latitude_meas":[v['Google_Coor'][0]],"longitude_meas":[v['Google_Coor'][1]]})
    df_store_lat_lng=df_store_lat_lng.append(df)
print(df_store_lat_lng.shape,df_store_lat_lng['location_id'].nunique())

(1422, 3) 1422
(1426, 3) 1426


In [11]:
df_store_lat_lng.head(2)

,location_id,latitude_meas,longitude_meas
0,1,39.913636,-82.914789
1,29,39.337172,-82.069765


In [12]:
df_all_store=df_all_store.rename(columns={"Location":"location_id"})
df_all_store=pd.merge(df_all_store,df_store_lat_lng,on="location_id",how="left")
print(df_all_store.shape,df_all_store['location_id'].nunique())

df_all_store[pd.isnull(df_all_store['latitude_meas'])]

(1424, 6) 1424


,location_id,Zip Code,SOTF,zip_cd,latitude_meas,longitude_meas


In [13]:
df_all_store=df_all_store.rename(columns={"zip_cd":"store_zip"})

In [14]:
TA_zips_for_all_stores=pd.DataFrame()

TA_zips_for_all_stores_1=pd.merge(df_all_store,df_zip_by_store,on="location_id",how="left")
TA_zips_for_all_stores_1.head(2)

,location_id,Zip Code,SOTF,store_zip,latitude_meas,longitude_meas,zip_cd,zip_type
0,5414,30518-3900,NaN,30518,34.093621,-84.000731,NaN,NaN
1,5413,35630-1511,NaN,35630,34.838159,-87.626999,NaN,NaN


In [15]:
TA_zips_for_all_stores_2=TA_zips_for_all_stores_1[pd.isnull(TA_zips_for_all_stores_1['zip_cd'])]
TA_zips_for_all_stores_1=TA_zips_for_all_stores_1[pd.notnull(TA_zips_for_all_stores_1['zip_cd'])]
print(TA_zips_for_all_stores_2.shape)

del TA_zips_for_all_stores_2['zip_cd']
del TA_zips_for_all_stores_2['zip_type']

(34, 8)


In [16]:
df_10_zips_all_new_stores=pd.DataFrame()
for ind, row in TA_zips_for_all_stores_2.iterrows():
    location_id=row['location_id']
    store_location=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_location,v,unit="mi")
        if dist<=10:
            # print(location_id,zip_cd)
            df=pd.DataFrame({"location_id":[location_id],"zip_cd":[zip_cd],"dist":[dist]})
            df_10_zips_all_new_stores=df_10_zips_all_new_stores.append(df)
df_10_zips_all_new_stores['zip_type']="zip_10"

In [17]:
print(df_10_zips_all_new_stores.shape,df_10_zips_all_new_stores['location_id'].nunique())

(833, 4) 34


In [18]:
df_new_store_zips=df_10_zips_all_new_stores[['location_id','zip_cd','zip_type']]
TA_zips_for_all_stores_2=pd.merge(TA_zips_for_all_stores_2,df_new_store_zips,how="left")
TA_zips_for_all_stores=TA_zips_for_all_stores_1.append(TA_zips_for_all_stores_2)
print(TA_zips_for_all_stores.shape,TA_zips_for_all_stores['location_id'].nunique(),TA_zips_for_all_stores['zip_cd'].nunique())

(23703, 8) 1424 13351


In [19]:
TA_zips_for_all_stores_1['zip_type'].nunique()

3

In [20]:
df_output_1=TA_zips_for_all_stores[TA_zips_for_all_stores['SOTF']=="YES"]
df_output_1.shape

(8611, 8)

In [21]:
TA_zips_for_all_stores.head(2)

,location_id,Zip Code,SOTF,store_zip,latitude_meas,longitude_meas,zip_cd,zip_type
4,1,43232-9301,YES,43232,39.913636,-82.914789,43227,P
5,1,43232-9301,YES,43232,39.913636,-82.914789,43207,P


In [22]:
TA_zips_for_all_stores.shape

(23703, 8)

In [23]:
df_output_2=TA_zips_for_all_stores.copy()
df_DMA_mapping_store=df_DMA_mapping.rename(columns={"zip_cd":"store_zip"})
df_output_2=pd.merge(df_output_2,df_DMA_mapping_store,on="store_zip",how="left").rename(columns={"DMA":"DMA_of_store"})
df_output_2=pd.merge(df_output_2,df_DMA_mapping,on="zip_cd",how="left").rename(columns={"DMA":"DMA_of_zip"})
print(df_output_2.shape)

(25436, 10)


In [24]:
df_output_2_all=df_output_2.groupby("DMA_of_store")['location_id','zip_cd'].nunique().reset_index()
df_output_2_all=df_output_2_all.rename(columns={"location_id":"all_store_count","zip_cd":"all_TA_zip_count"})

df_output_2_SOTF=df_output_2[df_output_2['SOTF']=="YES"]
df_output_2_SOTF=df_output_2_SOTF.groupby("DMA_of_store")['location_id','zip_cd'].nunique().reset_index()
df_output_2_SOTF=df_output_2_SOTF.rename(columns={"location_id":"SOTF_store_count","zip_cd":"SOTF_TA_zip_count"})

df_output_2=pd.merge(df_output_2_all,df_output_2_SOTF,on="DMA_of_store",how="outer")
df_output_2=df_output_2[['DMA_of_store','all_store_count','SOTF_store_count','all_TA_zip_count','SOTF_TA_zip_count']]
df_output_2=df_output_2.fillna(0)
df_output_2['Zip_Pctg_SOTF']=np.round(df_output_2['SOTF_TA_zip_count']/df_output_2['all_TA_zip_count'],4)



In [25]:
writer=pd.ExcelWriter("./BL_zips_by_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output_1.to_excel(writer,"request_1_zips_for_SOTF",index=False)

df_output_2.to_excel(writer,"request_2_by_DMA",index=False)

writer.save()

In [50]:
# Get the list of SOTF in multiple DMAs
df_output_1_qc=pd.merge(df_output_1,df_DMA_mapping_store,on="store_zip",how="left")
df_output_1_qc=df_output_1_qc.groupby("location_id")['DMA'].apply(set).to_frame().reset_index()
df_output_1_qc['DMA_count']=df_output_1_qc['DMA'].apply(len)
df_output_1_qc=df_output_1_qc.sort_values(["DMA_count",'location_id'],ascending=[False,True])
df_output_1_qc.shape

df_output_1_qc.to_csv("./BL_SOTF_in_multiple_DMAs_JL_"+str(datetime.datetime.now().date())+".csv",index=False)